## Tutorial 08 - Non linear Parabolic problem
**_Keywords: exact parametrized functions, POD-Galerkin_**

### 1. Introduction

In this tutorial, we consider the FitzHugh-Nagumo (F-N) system. The F-N system is used to describe neuron excitable systems. The nonlinear parabolic problem for the F-N system is defined on the interval $I=[0,L]$. Let $x\in I$, $t\geq0$

$$\begin{cases} 
    \varepsilon u_t(x,t) =\varepsilon^2u_{xx}(x,t)+g(u(x,t))-\omega(x,t)+c, & x\in I,\quad t\geq 0, \\
    \omega_t(x,t) =bu(x,t)-\gamma\omega(x,t)+c, & x\in I,\quad t\geq 0, \\
    u(x,0) = 0,\quad\omega(x,0)=0, & x\in I, \\
    u_x(0,t)=-i_0(t),\quad u_x(L,t)=0, & t\geq 0,
\end{cases}$$

where the nonlinear function is defined by
$$g(u) = u(u-0.1)(1-u)$$

and the parameters are given by $L = 1$, $\varepsilon = 0.015$, $b = 0.5$, $\gamma = 2$, and $c = 0.05$. The stimulus $i_0(t)=50000t^3\exp(-15t)$. The variables $u$ and $\omega$ represent the $\textit{voltage}$ and the $\textit{recovery of voltage}$, respectively. 

In order to obtain an exact solution of the problem we pursue a model reduction by means of a POD-Galerkin reduced order method.

### 2. Formulation for the F-N system

Let $u,\omega$ the solutions in the domain $I$.

For this problem we want to find $\boldsymbol{u}=(u,\omega)$ such that

$$
m\left(\partial_t\boldsymbol{u}(t),\boldsymbol{v}\right)+a\left(\boldsymbol{u}(t),\boldsymbol{v}\right)+c\left(u(t),v\right)=f(\boldsymbol{v})\quad \forall \boldsymbol{v}=(v,\tilde{v}), \text{ with }v,\tilde{v} \in\mathbb{V},\quad\forall t\geq0
$$



where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in L^2(I) : v|_{\{0\}}=0\}
$$
* the bilinear form $m(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$m(\partial\boldsymbol{u}(t), \boldsymbol{v})=\varepsilon\int_{I}\frac{\partial u}{\partial t}v \ d\boldsymbol{x} \ + \ \int_{I}\frac{\partial\omega}{\partial t}\tilde{v} \ d\boldsymbol{x},$$
* the bilinear form $a(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(\boldsymbol{u}(t), \boldsymbol{v})=\varepsilon^2\int_{I} \nabla u\cdot \nabla v \ d\boldsymbol{x}+\int_{I}\omega v \ d\boldsymbol{x} \ - \ b\int_{I} u\tilde{v} \ d\boldsymbol{x}+\gamma\int_{I}\omega\tilde{v} \ d\boldsymbol{x},$$
* the bilinear form $c(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$c(u, v)=-\int_{I} g(u)v \ d\boldsymbol{x},$$
* the linear form $f(\cdot): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(\boldsymbol{v})= c\int_{I}\left(v+\tilde{v}\right) \ d\boldsymbol{x} \ + \ \varepsilon^2i_0(t)\int_{\{0\}}v \ d\boldsymbol{s}.$$

The output of interest $s(t)$ is given by
$$s(t) = c\int_{I}\left[u(t)+\omega(t)\right] \ d\boldsymbol{x} \ + \ \varepsilon^2i_0(t)\int_{\{0\}}u(t) \ d\boldsymbol{s} $$.

In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

### 3. Affine Decomposition 

We set the variables $u:=u_1$, $\omega:=u_2$ and the test functions $v:=v_1$, $\tilde{v}:=v_2$.
For this problem the affine decomposition is straightforward:
    $$m(\boldsymbol{u},\boldsymbol{v})=\underbrace{\varepsilon}_{\Theta^{m}_0}\underbrace{\int_{I}u_1v_1 \ d\boldsymbol{x}}_{m_0(u_1,v_1)} \ + \ \underbrace{1}_{\Theta^{m}_1}\underbrace{\int_{I}u_2v_2 \ d\boldsymbol{x}}_{m_1(u_2,v_2)},$$
$$a(\boldsymbol{u},\boldsymbol{v})=\underbrace{\varepsilon^2}_{\Theta^{a}_0}\underbrace{\int_{I}\nabla u_1 \cdot \nabla v_1 \ d\boldsymbol{x}}_{a_0(u_1,v_1)} \ + \ \underbrace{1}_{\Theta^{a}_1}\underbrace{\int_{I}u_2v_1 \ d\boldsymbol{x}}_{a_1(u_2,v_1)} \ + \ \underbrace{-b}_{\Theta^{a}_2}\underbrace{\int_{I}u_1v_2 \ d\boldsymbol{x}}_{a_2(u_1,v_2)} \ + \ \underbrace{\gamma}_{\Theta^{a}_3}\underbrace{\int_{I}u_2v_2 \ d\boldsymbol{x}}_{a_3(u_2,v_2)},$$
$$c(u,v)=\underbrace{-1}_{\Theta^{c}_0}\underbrace{\int_{I}g(u_1)v_1 \ d\boldsymbol{x}}_{c_0(u_1,v_1)},$$
$$f(\boldsymbol{v}) = \underbrace{c}_{\Theta^{f}_0} \underbrace{\int_{I}(v_1 + v_2) \ d\boldsymbol{x}}_{f_0(v_1,v_2)} \ + \ \underbrace{\varepsilon^2i_0(t)}_{\Theta^{f}_1} \underbrace{\int_{\{0\}} v_1 \ d\boldsymbol{s}}_{f_1(v_1)}.$$
We will implement the numerical discretization of the problem in the class
```
class FitzHughNagumo(NonlinearParabolicProblem):
```
by specifying the coefficients $\Theta^{m}_*$, $\Theta^{a}_*$, $\Theta^{c}_*$ and $\Theta^{f}_*$ in the method
```
    def compute_theta(self, term):
```
and the bilinear forms $m_*(\boldsymbol{u}, \boldsymbol{v})$, $a_*(\boldsymbol{u}, \boldsymbol{v})$, $c_*(u, v)$ and linear forms $f_*(\boldsymbol{v})$ in
```
    def assemble_operator(self, term):
```

In [2]:
@ExactParametrizedFunctions()
class FitzHughNagumo(NonlinearParabolicProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NonlinearParabolicProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.du = TrialFunction(V)
        (self.du1, self.du2) = split(self.du)
        self.u = self._solution
        (self.u1, self.u2) = split(self.u)
        self.v = TestFunction(V)
        (self.v1, self.v2) = split(self.v)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Problem coefficients
        self.epsilon = 0.015
        self.b = 0.5
        self.gamma = 2
        self.c = 0.05
        self.i0 = lambda t: 50000 * t**3 * exp(-15 * t)
        self.g = lambda v: v * (v - 0.1) * (1 - v)
        # Customize time stepping parameters
        self._time_stepping_parameters.update({
            "report": True,
            "snes_solver": {
                "linear_solver": "umfpack",
                "maximum_iterations": 20,
                "report": True
            }
        })

    # Return custom problem name
    def name(self):
        return "FitzHughNagumoExact"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    def compute_theta(self, term):
        if term == "m":
            theta_m0 = self.epsilon
            theta_m1 = 1.
            return (theta_m0, theta_m1)
        elif term == "a":
            theta_a0 = self.epsilon**2
            theta_a1 = 1.
            theta_a2 = - self.b
            theta_a3 = self.gamma
            return (theta_a0, theta_a1, theta_a2, theta_a3)
        elif term == "c":
            theta_c0 = - 1.
            return (theta_c0,)
        elif term == "f":
            t = self.t
            theta_f0 = self.c
            theta_f1 = self.epsilon**2 * self.i0(t)
            return (theta_f0, theta_f1)
        elif term == "s":
            t = self.t
            theta_s0 = self.c
            theta_s1 = self.epsilon**2 * self.i0(t)
            return (theta_s0, theta_s1)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    def assemble_operator(self, term):
        (v1, v2) = (self.v1, self.v2)
        dx = self.dx
        if term == "m":
            (u1, u2) = (self.du1, self.du2)
            m0 = u1 * v1 * dx
            m1 = u2 * v2 * dx
            return (m0, m1)
        elif term == "a":
            (u1, u2) = (self.du1, self.du2)
            a0 = inner(grad(u1), grad(v1)) * dx
            a1 = u2 * v1 * dx
            a2 = u1 * v2 * dx
            a3 = u2 * v2 * dx
            return (a0, a1, a2, a3)
        elif term == "c":
            u1 = self.u1
            c0 = self.g(u1) * v1 * dx
            return (c0,)
        elif term == "f":
            ds = self.ds
            f0 = v1 * dx + v2 * dx
            f1 = v1 * ds(1)
            return (f0, f1)
        elif term == "s":
            (v1, v2) = (self.v1, self.v2)
            ds = self.ds
            s0 = v1 * dx + v2 * dx
            s1 = v1 * ds(1)
            return (s0, s1)
        elif term == "inner_product":
            (u1, u2) = (self.du1, self.du2)
            x0 = inner(grad(u1), grad(v1)) * dx + u2 * v2 * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NonlinearParabolicProblem)
def CustomizeReducedNonlinearParabolic(ReducedNonlinearParabolic_Base):
    class ReducedNonlinearParabolic(ReducedNonlinearParabolic_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNonlinearParabolic_Base.__init__(self, truth_problem, **kwargs)
            self._time_stepping_parameters.update({
                "report": True,
                "nonlinear_solver": {
                    "report": True,
                    "line_search": "wolfe"
                }
            })

    return ReducedNonlinearParabolic

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/interval.xml")
subdomains = MeshFunction("size_t", mesh, "data/interval_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/interval_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = VectorFunctionSpace(mesh, "Lagrange", 1, dim=2)

### 4.3. Allocate an object of the FitzHughNagumo class

In [5]:
problem = FitzHughNagumo(V, subdomains=subdomains, boundaries=boundaries)
mu_range = []
problem.set_mu_range(mu_range)
problem.set_time_step_size(0.02)
problem.set_final_time(8)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
reduction_method.initialize_training_set(1)
reduced_problem = reduction_method.offline()

=            FitzHughNagumoExact POD-Galerkin offline phase begins             =

###################################### 0 #######################################
truth solve for mu = ()
# t = 0.02
  0 SNES Function norm 2.210911e-03
  1 SNES Function norm 1.063340e-04
  2 SNES Function norm 6.047399e-07
  3 SNES Function norm 1.982447e-11
# t = 0.04
  0 SNES Function norm 2.849024e-03
  1 SNES Function norm 1.010348e-04
  2 SNES Function norm 5.953550e-07
  3 SNES Function norm 2.184160e-11
# t = 0.06
  0 SNES Function norm 3.734542e-03
  1 SNES Function norm 1.217784e-04
  2 SNES Function norm 7.911830e-07
  3 SNES Function norm 3.547206e-11
# t = 0.08
  0 SNES Function norm 4.968389e-03
  1 SNES Function norm 1.423033e-04
  2 SNES Function norm 3.604419e-07
  3 SNES Function norm 2.124304e-12
# t = 0.1
  0 SNES Function norm 6.954749e-03
  1 SNES Function norm 5.282859e-04
  2 SNES Function norm 3.852334e-05
  3 SNES Function norm 2.557857e-07
  4 SNES Function norm 1.124143e-11
# t

  1 SNES Function norm 4.941275e-04
  2 SNES Function norm 1.393754e-05
  3 SNES Function norm 1.507308e-08
  4 SNES Function norm 2.446187e-14
# t = 1.18
  0 SNES Function norm 8.289758e-03
  1 SNES Function norm 8.628667e-04
  2 SNES Function norm 3.159145e-05
  3 SNES Function norm 4.864398e-08
  4 SNES Function norm 1.192499e-13
# t = 1.2
  0 SNES Function norm 9.860113e-03
  1 SNES Function norm 5.254171e-04
  2 SNES Function norm 9.651473e-06
  3 SNES Function norm 3.557874e-09
  4 SNES Function norm 5.085819e-16
# t = 1.22
  0 SNES Function norm 1.058966e-02
  1 SNES Function norm 1.655877e-04
  2 SNES Function norm 8.540870e-07
  3 SNES Function norm 2.380560e-11
# t = 1.24
  0 SNES Function norm 1.078533e-02
  1 SNES Function norm 3.452413e-05
  2 SNES Function norm 3.516280e-08
  3 SNES Function norm 3.765527e-14
# t = 1.26
  0 SNES Function norm 1.066145e-02
  1 SNES Function norm 4.044651e-06
  2 SNES Function norm 4.802536e-10
# t = 1.28
  0 SNES Function norm 1.034985e-02

  3 SNES Function norm 8.844135e-13
# t = 2.42
  0 SNES Function norm 1.073063e-02
  1 SNES Function norm 1.221650e-05
  2 SNES Function norm 4.661434e-09
  3 SNES Function norm 9.135288e-16
# t = 2.44
  0 SNES Function norm 1.049722e-02
  1 SNES Function norm 7.248829e-07
  2 SNES Function norm 1.678495e-11
# t = 2.46
  0 SNES Function norm 1.012817e-02
  1 SNES Function norm 7.071741e-07
  2 SNES Function norm 2.560734e-11
# t = 2.48
  0 SNES Function norm 9.693762e-03
  1 SNES Function norm 1.821018e-06
  2 SNES Function norm 9.966741e-11
# t = 2.5
  0 SNES Function norm 9.234410e-03
  1 SNES Function norm 2.699737e-06
  2 SNES Function norm 2.069940e-10
# t = 2.52
  0 SNES Function norm 8.771481e-03
  1 SNES Function norm 3.184799e-06
  2 SNES Function norm 2.916701e-10
# t = 2.54
  0 SNES Function norm 8.315724e-03
  1 SNES Function norm 3.405407e-06
  2 SNES Function norm 3.411197e-10
# t = 2.56
  0 SNES Function norm 7.872322e-03
  1 SNES Function norm 3.476883e-06
  2 SNES Func

  1 SNES Function norm 2.687039e-06
  2 SNES Function norm 2.452427e-10
# t = 3.68
  0 SNES Function norm 5.071785e-03
  1 SNES Function norm 2.601788e-06
  2 SNES Function norm 2.351534e-10
# t = 3.7
  0 SNES Function norm 4.762598e-03
  1 SNES Function norm 2.519831e-06
  2 SNES Function norm 2.255158e-10
# t = 3.72
  0 SNES Function norm 4.466257e-03
  1 SNES Function norm 2.441382e-06
  2 SNES Function norm 2.163703e-10
# t = 3.74
  0 SNES Function norm 4.182234e-03
  1 SNES Function norm 2.366535e-06
  2 SNES Function norm 2.077341e-10
# t = 3.76
  0 SNES Function norm 3.910018e-03
  1 SNES Function norm 2.295326e-06
  2 SNES Function norm 1.996132e-10
# t = 3.78
  0 SNES Function norm 3.649111e-03
  1 SNES Function norm 2.227767e-06
  2 SNES Function norm 1.920080e-10
# t = 3.8
  0 SNES Function norm 3.399032e-03
  1 SNES Function norm 2.163859e-06
  2 SNES Function norm 1.849174e-10
# t = 3.82
  0 SNES Function norm 3.159312e-03
  1 SNES Function norm 2.103613e-06
  2 SNES Funct

  2 SNES Function norm 3.141042e-10
# t = 4.96
  0 SNES Function norm 3.389227e-03
  1 SNES Function norm 2.746976e-06
  2 SNES Function norm 3.134377e-10
# t = 4.98
  0 SNES Function norm 3.133617e-03
  1 SNES Function norm 2.720095e-06
  2 SNES Function norm 3.144441e-10
# t = 5
  0 SNES Function norm 2.887305e-03
  1 SNES Function norm 2.702080e-06
  2 SNES Function norm 3.174050e-10
# t = 5.02
  0 SNES Function norm 2.649721e-03
  1 SNES Function norm 2.694078e-06
  2 SNES Function norm 3.226931e-10
# t = 5.04
  0 SNES Function norm 2.420286e-03
  1 SNES Function norm 2.697567e-06
  2 SNES Function norm 3.308106e-10
# t = 5.06
  0 SNES Function norm 2.198410e-03
  1 SNES Function norm 2.714481e-06
  2 SNES Function norm 3.424453e-10
# t = 5.08
  0 SNES Function norm 1.983484e-03
  1 SNES Function norm 2.747378e-06
  2 SNES Function norm 3.585569e-10
# t = 5.1
  0 SNES Function norm 1.774883e-03
  1 SNES Function norm 2.799687e-06
  2 SNES Function norm 3.805123e-10
# t = 5.12
  0 S

  0 SNES Function norm 7.782276e-04
  1 SNES Function norm 1.716806e-06
  2 SNES Function norm 1.502685e-10
# t = 6.24
  0 SNES Function norm 6.429271e-04
  1 SNES Function norm 1.734417e-06
  2 SNES Function norm 1.557647e-10
# t = 6.26
  0 SNES Function norm 5.138715e-04
  1 SNES Function norm 1.764852e-06
  2 SNES Function norm 1.637433e-10
# t = 6.28
  0 SNES Function norm 3.930048e-04
  1 SNES Function norm 1.811234e-06
  2 SNES Function norm 1.750347e-10
# t = 6.3
  0 SNES Function norm 2.862412e-04
  1 SNES Function norm 1.877891e-06
  2 SNES Function norm 1.908894e-10
# t = 6.32
  0 SNES Function norm 2.115161e-04
  1 SNES Function norm 1.970953e-06
  2 SNES Function norm 2.132482e-10
# t = 6.34
  0 SNES Function norm 2.051452e-04
  1 SNES Function norm 2.099284e-06
  2 SNES Function norm 2.452301e-10
# t = 6.36
  0 SNES Function norm 2.728558e-04
  1 SNES Function norm 2.276038e-06
  2 SNES Function norm 2.920520e-10
# t = 6.38
  0 SNES Function norm 3.795304e-04
  1 SNES Func

# t = 7.48
  0 SNES Function norm 1.111226e-03
  1 SNES Function norm 1.618218e-05
  2 SNES Function norm 1.482129e-08
  3 SNES Function norm 1.292217e-14
# t = 7.5
  0 SNES Function norm 1.557795e-03
  1 SNES Function norm 2.464284e-05
  2 SNES Function norm 2.999391e-08
  3 SNES Function norm 4.486534e-14
# t = 7.52
  0 SNES Function norm 2.190919e-03
  1 SNES Function norm 3.687884e-05
  2 SNES Function norm 4.168228e-08
  3 SNES Function norm 5.960673e-14
# t = 7.54
  0 SNES Function norm 3.164184e-03
  1 SNES Function norm 3.316425e-05
  2 SNES Function norm 6.452740e-09
  3 SNES Function norm 1.065322e-15
# t = 7.56
  0 SNES Function norm 4.734858e-03
  1 SNES Function norm 2.065944e-04
  2 SNES Function norm 3.718031e-06
  3 SNES Function norm 1.589718e-09
  4 SNES Function norm 3.753645e-16
# t = 7.58
  0 SNES Function norm 6.962570e-03
  1 SNES Function norm 7.644089e-04
  2 SNES Function norm 2.775253e-05
  3 SNES Function norm 4.086948e-08
  4 SNES Function norm 9.473639e-14

#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PINN}(X_{PINN}; W) = \frac1n \sum_{i=1}^n \left\|M(\boldsymbol{\mu^{(i)}})\frac{\partial \operatorname{N}_W}{\partial t}(\boldsymbol{\mu^{(i)}}) + A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)}) + \boldsymbol{c}(\boldsymbol{\mu}^{(i)})\right\|_2^2$$

over $W$, where for a given $\boldsymbol{\mu}$, $M(\boldsymbol{\mu})$ is the mass matrix corresponding to the bilinear form $m$, $A(\boldsymbol{\mu})$ is the assembled matrix corresponding to the bilinear form $a$, $\boldsymbol{f}(\boldsymbol{\mu})$ is the assembled vector corresponding to the linear form $f$, and $\boldsymbol{c}(\boldsymbol{\mu})$ is a vector corresponding to the nonlinear form $c$. The partial derivative $\frac{\partial \operatorname{N}_W}{\partial t}(\boldsymbol{\mu^{(i)}})$ is computed via automatic differentiation in PyTorch.

In [8]:
input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization()

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=1)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99999)

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [15]:
#optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
#pinn_trainer.optimizer = optimizer
pinn_trainer.train()

10000 188.4112752683634
10100 188.15667921694387
10200 187.8359661165562
10300 187.55178493606206
10400 187.30465048209922
10500 187.0460644728543
10600 187.10491065228445
10700 186.56744590967696
10800 186.32741642733882
10900 187.84642105538188
11000 185.88073393653318
11100 185.65236534392048
11200 185.43034758902033
11300 185.1960614581435
11400 184.95385847097026
11500 185.176515542313
11600 184.49554624856822
11700 184.25746982862907
11800 184.0122274297807
11900 183.81456593448405
12000 183.5428549204832
12100 183.33784746711174
12200 183.0990678437015
12300 182.9926457303177
12400 182.67550902245495
12500 182.46376636485533
12600 182.26513654368839
12700 182.06604664563395
12800 181.8797769248185
12900 181.6682462152896
13000 181.46294957149655
13100 182.4621906687575
13200 181.02612000322176
13300 180.78245195000923
13400 180.52158476573052
13500 180.23497491135242
13600 179.9163360015489
13700 179.58436184143693
13800 179.2126775322014
13900 178.82486033415017
14000 178.43346

In [ ]:
pinn_loss.operators['c'][0]

In [ ]:
from rbnics.backends.dolfin.evaluate import evaluate
problem.set_time(0.)
evaluate(problem.operator['c'][0])[:]

In [ ]:
pinn_net.time_augmented_mu.shape, pinn_loss.operators['c'].shape, 51*80

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.StandardNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W),$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [10]:
reduction_method.initialize_testing_set(1)
#reduction_method.error_analysis()

False

#### 4.6.2 PINN Error Analysis

In [11]:
test_mu = torch.tensor(reduction_method.testing_set)

In [14]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn, relative=False
)

#################################### N = 20 ####################################
ERROR	NN-HF			NN-RO			RO-HF
min	0.43835376496167455	0.43835376496167455	0.0
mean	1.0287790165310742	1.028777778435999	0.00031566331767224744
max	2.039400104290944	2.0393977832859855	0.0013751011155775043


In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, tuple(), input_normalization_pinn, output_normalization_pinn, t=0, component=0
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, tuple(), input_normalization_pdnn, output_normalization_pdnn, t=0, component=0
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, tuple(), input_normalization_prnn, output_normalization_prnn, t=0, component=0
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, relative=False
)

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()